In [1]:
import time
import cv2
import numpy as np
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
cap = cv2.VideoCapture(r"C:\Users\MATANGEE\Downloads\knee_exercise_video.mp4")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('Output.mp4',cv2.VideoWriter_fourcc(*'DIVX'),25,(width,height))

#Variables
stage = None
bend_time = 0
reps = 0
count_reps = False
feedback_message = None

def calculate_angle(a,b,c):
    a = np.array(a) #First
    b = np.array(b) #Mid
    c = np.array(c) #End 
    radians = np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle>180.0:
        angle = 360-angle
    return angle

## setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        #Recolor Image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        #Make detection
        results = pose.process(image)
    
        #Recoloring back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        #Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            #Customize the coordinates of you want to have in your exercise
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            
            #Calculate angle according to your exercise
            angle = calculate_angle(hip,knee,ankle)
            
            #Visualize angle(Set Parameters)
            cv2.putText(image,str(angle),
                        tuple(np.multiply(knee,[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,
                             0.5,(255,255,255),2,cv2.LINE_AA)
            
            #Curl counter logic (set it different for different exercise )
            if angle >140:
                stage = 'leg straight'
                count_reps = True
                t0 = time.time()
            if angle < 140 :
                if stage != "knee bend" :
                    t0 = time.time()
                    bend_time = 0
                stage = "knee bend"
                bend_time = time.time() - t0
                if bend_time > 8 and count_reps:
                    reps += 1
                    count_reps = False
                    feedback_message = ""
                if bend_time<8:
                    feedback_message = "Keep your knee bend"

        except:
            pass
        
        #Setup status box
        cv2.rectangle(image, (0,0),(690,54),(0,0,0),-1)
        
        #Rep data
        cv2.putText(image,"REP",(15,16),cv2.FONT_HERSHEY_SIMPLEX,
                             0.5,(0,255,255),1,cv2.LINE_AA)
        cv2.putText(image,str(reps),(15,45),cv2.FONT_HERSHEY_SIMPLEX,
                             1,(255,255,0),2,cv2.LINE_AA)
        
        #Stage data
        cv2.putText(image,'Stage',(60,16),cv2.FONT_HERSHEY_SIMPLEX,
                             0.5,(0,255,255),1,cv2.LINE_AA)
        cv2.putText(image,stage,(60,45),cv2.FONT_HERSHEY_SIMPLEX,
                             1,(255,255,0),2,cv2.LINE_AA)
        #Timer data
        cv2.putText(image,'Timer',(270,16),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),1,cv2.LINE_AA)
        cv2.putText(image,"{:.1f}".format(bend_time),(270,45),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2,cv2.LINE_AA)
        
        #Feedback data
        cv2.putText(image,'Feedback',(350,16),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),1,cv2.LINE_AA)
        cv2.putText(image,feedback_message,(350,45),cv2.FONT_HERSHEY_SIMPLEX,
                             1,(255,255,0),2,cv2.LINE_AA)
        
    
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        writer.write(image)
        cv2.imshow("Mediapipe Feed",image)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    cap.release()
    writer.release()
    cv2.destroyAllWindows()